<a href="https://colab.research.google.com/github/Loop3D/2022_iamg_workshop/blob/main/map2loop/2b_Building_a_model_from_local_sources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2b Building a model from local data sources

## Installing dependencies
This notebook will use two libraries from the Loop project
1. LoopStructural - https://github.com/Loop3D/LoopStructural
2. map2loop - https://github.com/Loop3D/map2loop-2


The following code blocks will install the required dependencies for the Loop libraries into this colab environment. 

In this notebook, the same code as the previous notebook is used, except that this time we are using local data sources (shapefiles for the geological data and a geotiff for the Digital Terrain Model)

In [ ]:
! git clone https://github.com/Loop3D/map2loop2-notebooks

In [ ]:
!pip install rasterio
!pip install git+https://github.com/geopandas/geopandas.git@v0.10.2
!pip install hjson
!pip install owslib
!pip install git+https://github.com/Loop3D/map2model_cpp.git
!pip install git+https://github.com/Loop3D/LoopProjectFile.git
!pip install pygeos
!pip install mplstereonet
!pip install lavavu-osmesa
!pip install LoopStructural
!pip install git+https://github.com/Loop3D/map2loop-2.git
!pip install beartype

In [ ]:
import os
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel
import time

from datetime import datetime
nowtime=datetime.now().isoformat(timespec='minutes')   
model_name=nowtime.replace(":","-").replace("T","-")

t0 = time.time()

proj = Project( 
                  geology_filename="./map2loop2-notebooks/source_data/geol_clip.shp",
                  fault_filename="./map2loop2-notebooks/source_data/faults_clip.shp",
                  fold_filename="./map2loop2-notebooks/source_data/folds_clip.shp",
                  structure_filename="./map2loop2-notebooks/source_data/structure_clip.shp",
                  mindep_filename="./map2loop2-notebooks/source_data/mindeps_clip.shp",
                  dtm_filename='./map2loop2-notebooks/source_data/dtm_rp.tif',
                  metadata_filename='./map2loop2-notebooks/source_data/example.hjson',
                  overwrite="true",
                  verbose_level=VerboseLevel.NONE,
                  project_path=model_name,
                  working_projection="EPSG:28350",
                )

proj.update_config(
                    out_dir=model_name,
                    bbox_3d={
                         "minx": 520000,
                         "miny": 7490000,
                         "maxx": 550000,
                         "maxy": 7510000,
                         "base": -3200,
                         "top": 1200,
                     },
                     run_flags={                        
                        'aus': True,
                        'close_dip': -999,
                        'contact_decimate': 5,
                        'contact_dip': -999,
                        'contact_orientation_decimate': 5,
                        'deposits': "Fe,Cu,Au,NONE",
                        'dist_buffer': 10,
                        'dtb': '',
                        'fat_step': 750,
                        'fault_decimate': 5,
                        'fault_dip': 90,
                        'fold_decimate': 5,
                        'interpolation_scheme': 'scipy_rbf',
                        'interpolation_spacing': 500,
                        'intrusion_mode': 0,
                        'max_thickness_allowed': 10000,
                        'min_fault_length': 5000,
                        'misorientation': 30,
                        'null_scheme': 'null',
                        'orientation_decimate': 0,
                        'pluton_dip': 45,
                        'pluton_form': 'saucers',
                        'thickness_buffer': 5000,
                        'use_fat': False,
                        'use_interpolations': False,
                        'fault_orientation_clusters':2,
                        'fault_length_clusters':2
                    },
                    proj_crs= 'EPSG:28350',
                    clut_path='./map2loop2-notebooks/source_data/500kibg_colours.csv',
                    #quiet='all' # change this to 'None' (with quotes) to see intermediate output
                  )
proj.config.c_l['intrusive']='mafic intrusive'
proj.run()

In [ ]:
# Define project pathing from m2l
proj_path = proj.config.project_path
graph_path = proj.config.graph_path
tmp_path = proj.config.tmp_path
output_path = proj.config.output_path

# Define project bounds
minx,miny,maxx,maxy = proj.config.bbox
model_base = proj.config.bbox_3d['base']
model_top = proj.config.bbox_3d['top']

dtm_path=model_name+'/dtm/'
vtk_path=model_name+'/vtk/'

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

from LoopStructural import GeologicalModel
from LoopStructural.visualisation import LavaVuModelViewer
from datetime import datetime
import os
import time
import shutil
import logging
#logging.getLogger().setLevel(logging.ERROR)
import lavavu
import numpy as np
from map2loop.m2l_utils import save_dtm_mesh

t1 = time.time()

filename=vtk_path+'/'+'surface_name_{}.vtk'

save_dtm_mesh(dtm_path,vtk_path)

f=open(tmp_path+'/bbox.csv','w')
f.write('minx,miny,maxx,maxy,lower,upper\n')
ostr='{},{},{},{},{},{}\n'.format(minx,miny,maxx,maxy,model_base,model_top)
f.write(ostr)
f.close()


fault_params = {'interpolatortype':'FDI',
                 'nelements':1e5,
                }
foliation_params = {'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
                    'nelements':1e5,  # how many tetras/voxels
                    }
model, m2l_data = GeologicalModel.from_map2loop_directory(proj_path,
                                                          fault_params=fault_params,
                                                          rescale=False,
                                                          foliation_params=foliation_params)
model.update()
#model.to_file(output_path + "/model.pickle")    

view = LavaVuModelViewer(model,vertical_exaggeration=1) 
view.nsteps=np.array([50,50,50])
for sg in model.feature_name_index:
    if( 'super' in sg):
        view.add_data(model.features[model.feature_name_index[sg]])
view.nelements = 1e5
view.add_model_surfaces(filename=filename,faults=False)
view.nelements=1e6
view.add_model_surfaces(filename=filename,strati=False,displacement_cmap = 'rainbow')
view.lv.webgl(vtk_path+model_name)
view.nsteps = np.array([200,200,200])

view.add_model()


t2 = time.time()
print("m2l",(t1-t0)/60.0,"LoopStructural",(t2-t1)/60.0,"Total",(t2-t0)/60.0,"minutes")

In [ ]:
view.interactive()  
